In [138]:
from torchvision import datasets
from torchvision.transforms import ToTensor

In [139]:
train_data = datasets.MNIST(
    root = 'data',
    train = True,
    transform = ToTensor(),
    download = True,
)
test_data = datasets.MNIST(
    root = 'data',
    train = False,
    transform = ToTensor(),
    download = True,
)

In [140]:
train_data

Dataset MNIST
    Number of datapoints: 60000
    Root location: data
    Split: Train
    StandardTransform
Transform: ToTensor()

In [141]:
test_data

Dataset MNIST
    Number of datapoints: 10000
    Root location: data
    Split: Test
    StandardTransform
Transform: ToTensor()

In [142]:
train_data.data.shape

torch.Size([60000, 28, 28])

In [143]:
test_data.data.shape

torch.Size([10000, 28, 28])

In [144]:
train_data.targets.size()

torch.Size([60000])

In [145]:
train_data.targets

tensor([5, 0, 4,  ..., 5, 6, 8])

In [146]:
from torch.utils.data import DataLoader

loaders = {
    'train' : DataLoader(train_data,
                         batch_size=100,
                         shuffle=True,
                         num_workers=1),

    'test' : DataLoader(test_data,
                         batch_size=100,
                         shuffle=True,
                         num_workers=1),
}

In [147]:
loaders

{'train': <torch.utils.data.dataloader.DataLoader at 0x79e53e0e90f0>,
 'test': <torch.utils.data.dataloader.DataLoader at 0x79e53e0e95a0>}

In [148]:
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

class CNN(nn.Module):

    def __init__(self):
        super(CNN, self).__init__()

        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x),2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)),2))
        x = x.view(-1, 320)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)

        return F.log_softmax(x)

In [149]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = CNN().to(device)

optimizer = optim.Adam(model.parameters(), lr=0.001)

loss_fn = nn.CrossEntropyLoss()

def train(epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(loaders['train']):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = loss_fn(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 20 == 0:
            print(f'Train Epoch: {epoch} [{batch_idx * len(data)}/{len(loaders["train"].dataset)} ({100. * batch_idx / len(loaders["train"]):.0f}%)]\tLoss: {loss.item():.6f}')

def test():
  model.eval()

  test_loss = 0
  correct = 0

  with torch.no_grad():
    for data, target in loaders['test']:
      data, target = data.to(device), target.to(device)
      output = model(data)
      test_loss += loss_fn(output, target).item()
      pred = output.data.max(1, keepdim=True)
      correct += pred.eq(target.view_as(pred)).sum().item()

  test_loss /= len(loaders['test'].dataset)
  print(f'\nTest set: Average loss: {test_loss:.4f}, Accuracy: {correct}/{len(loaders["test"].dataset)} ({100. * correct / len(loaders["test"].dataset):.0f}%)\n')


In [150]:
for epoch in range(1, 11):
    train(epoch)
    test()

<ipython-input-148-3ed9f7880673>:24: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)


Train Epoch: 1 [0/60000 (0%)]	Loss: 2.320102
Train Epoch: 1 [2000/60000 (3%)]	Loss: 2.230495
Train Epoch: 1 [4000/60000 (7%)]	Loss: 1.798131
Train Epoch: 1 [6000/60000 (10%)]	Loss: 1.348101
Train Epoch: 1 [8000/60000 (13%)]	Loss: 1.184478
Train Epoch: 1 [10000/60000 (17%)]	Loss: 0.820484
Train Epoch: 1 [12000/60000 (20%)]	Loss: 0.605767
Train Epoch: 1 [14000/60000 (23%)]	Loss: 0.522360
Train Epoch: 1 [16000/60000 (27%)]	Loss: 0.581640
Train Epoch: 1 [18000/60000 (30%)]	Loss: 0.610064
Train Epoch: 1 [20000/60000 (33%)]	Loss: 0.581374
Train Epoch: 1 [22000/60000 (37%)]	Loss: 0.422066
Train Epoch: 1 [24000/60000 (40%)]	Loss: 0.505043
Train Epoch: 1 [26000/60000 (43%)]	Loss: 0.541472
Train Epoch: 1 [28000/60000 (47%)]	Loss: 0.603349
Train Epoch: 1 [30000/60000 (50%)]	Loss: 0.437095
Train Epoch: 1 [32000/60000 (53%)]	Loss: 0.247183
Train Epoch: 1 [34000/60000 (57%)]	Loss: 0.358359
Train Epoch: 1 [36000/60000 (60%)]	Loss: 0.445260
Train Epoch: 1 [38000/60000 (63%)]	Loss: 0.378149
Train Epoch

AttributeError: 'torch.return_types.max' object has no attribute 'eq'

In [ ]:
device

In [ ]:
import matplotlib.pyplot as plt

model.eval()

data, target = test_data[4]

data= data.unsqueeze(0).to(device)

output = model(data)

prediction = output.argmax(dim=1, keepdim=True).item()

print(f'{prediction}')

image = data.squeeze(0).squeeze(0).cpu().numpy()

plt.imshow(image, cmap='gray')
plt.show()